In [2]:
%matplotlib widget

import logging

import httpx
import sqlmodel

from arpav_ppcv import (
    config,
    database,
)
from arpav_ppcv.observations_harvester import operations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))
client = httpx.Client()

In [4]:
tdd_var = database.get_variable_by_name(session, "TDd")

In [7]:
remote_tdd_stations_gen = operations.fetch_remote_stations(
    client, 
    [tdd_var], 
    fetch_stations_with_months=True, 
    fetch_stations_with_seasons=True,
    fetch_stations_with_yearly_measurements=True
)

In [8]:
remote_tdd_stations = list(remote_tdd_stations_gen)

INFO:arpav_ppcv.observations_harvester.operations:Retrieving stations with monthly measurements for variable 'TDd'...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 1...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 2...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 3...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 4...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 5...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 6...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 7...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 8...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 9...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 10...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 11...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 12...
INFO:arpav_ppcv

In [14]:
len(set([(i["statcd"], i["statnm"]) for i in remote_tdd_stations]))

136

In [15]:
all_vars = database.collect_all_variables(session)

In [17]:
raw_stations = {}
for obs_var in all_vars:
    raw_stations[obs_var.name] = list(
        operations.fetch_remote_stations(
            client,
            [obs_var],
            fetch_stations_with_months=True,
            fetch_stations_with_seasons=True,
            fetch_stations_with_yearly_measurements=True
        )
    )

INFO:arpav_ppcv.observations_harvester.operations:Retrieving stations with monthly measurements for variable 'FD'...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 1...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 2...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 3...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 4...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 5...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 6...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 7...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 8...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 9...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 10...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 11...
INFO:arpav_ppcv.observations_harvester.operations:Processing month 12...
INFO:arpav_ppcv.

In [22]:
parsed = {
    k: set([(i["statcd"], i["statnm"]) for i in v])
    for k, v in raw_stations.items()
}

In [39]:
for name, items in parsed.items():
    print(name)
    for i in sorted(list(items)):
        print(f"\t{i[0]} ({i[1]})")
    print("---")

FD
	3 (Arabba)
	9 (Caprile)
	11 (Malga Ciapela)
	15 (Gares)
	19 (Agordo)
	21 (Sant'Andrea (Gosaldo))
	22 (Sant'Antonio Tortal)
	25 (Sospirolo)
	28 (Monte Cesen)
	37 (Passo Falzarego)
	41 (Marmolada - Punta Rocca)
	47 (Faloria)
	48 (Podestagno (Cortina d'Ampezzo))
	50 (Villanova (Borca di Cadore))
	53 (Auronzo)
	55 (Pian del Crep (Val di Zoldo))
	56 (Forno di Zoldo - Campo)
	58 (Santo Stefano di Cadore)
	59 (Domegge di Cadore)
	60 (Malga Campobon (San Pietro di Cadore))
	61 (Cimacanale (Santo Stefano di Cadore))
	67 (Monte Avena)
	68 (Castana)
	71 (San Giovanni Ilarione)
	72 (Astico a Pedescala)
	73 (Contra' Doppio (Posina))
	74 (Monte Verena)
	76 (Turcati (Recoaro Terme))
	77 (Recoaro Mille)
	79 (Valdagno)
	80 (Passo Pordoi)
	81 (Monte Summano)
	82 (Pove del Grappa - Costalunga)
	83 (Montecchio Precalcino)
	87 (San Bortolo)
	88 (Crespadoro)
	91 (Passo Monte Croce Comelico)
	92 (Col Indes (Tambre))
	93 (Torch)
	96 (Bagnolo di Po - Pellizzare)
	98 (Concadirame (Rovigo))
	99 (San Bellino)

In [25]:
{k: len(v) for k,v in parsed.items()}

{'FD': 136,
 'PRCPTOT': 136,
 'SU30': 136,
 'TDd': 136,
 'TNd': 136,
 'TR': 136,
 'TXd': 136}

In [27]:
unique_stations = set()
for var_name, raw_station_list in raw_stations.items():
    for raw_station in raw_station_list:
        relevant = (raw_station["statcd"], raw_station["statnm"])
        unique_stations.add(relevant)

In [35]:
for code, name in sorted(list(unique_stations)):
    print(f"{code} ({name})")

3 (Arabba)
9 (Caprile)
11 (Malga Ciapela)
15 (Gares)
19 (Agordo)
21 (Sant'Andrea (Gosaldo))
22 (Sant'Antonio Tortal)
25 (Sospirolo)
28 (Monte Cesen)
37 (Passo Falzarego)
41 (Marmolada - Punta Rocca)
47 (Faloria)
48 (Podestagno (Cortina d'Ampezzo))
50 (Villanova (Borca di Cadore))
53 (Auronzo)
55 (Pian del Crep (Val di Zoldo))
56 (Forno di Zoldo - Campo)
58 (Santo Stefano di Cadore)
59 (Domegge di Cadore)
60 (Malga Campobon (San Pietro di Cadore))
61 (Cimacanale (Santo Stefano di Cadore))
67 (Monte Avena)
68 (Castana)
71 (San Giovanni Ilarione)
72 (Astico a Pedescala)
73 (Contra' Doppio (Posina))
74 (Monte Verena)
76 (Turcati (Recoaro Terme))
77 (Recoaro Mille)
79 (Valdagno)
80 (Passo Pordoi)
81 (Monte Summano)
82 (Pove del Grappa - Costalunga)
83 (Montecchio Precalcino)
87 (San Bortolo)
88 (Crespadoro)
91 (Passo Monte Croce Comelico)
92 (Col Indes (Tambre))
93 (Torch)
96 (Bagnolo di Po - Pellizzare)
98 (Concadirame (Rovigo))
99 (San Bellino)
100 (Conegliano)
101 (Porto Tolle - Pradon)
